In [ ]:
# To perform operations on dataset
import pandas as pd
import numpy as np

# Machine learning model
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

### To remove the duplicate enteriess in the training dataset.

In [ ]:
with open('/content/Phising_Training_Dataset.csv','r') as in_file, open('ouput.csv','w') as out_file:
    seen = set() # set for fast O(1) amortized lookup
    for line in in_file:
        if line in seen: 
          continue # skip duplicate
        seen.add(line)
        out_file.write(line)

### Modeling begins

In [ ]:
df = pd.read_csv('/content/ouput.csv')
df_test = pd.read_csv('/content/Phising_Testing_Dataset.csv')

In [ ]:
X = df.iloc[:, :-1]

y = df.iloc[:, -1]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=0)

In [ ]:
seed = 5
kfold = model_selection.RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state = seed)

# initialize the base classifier
base_cls = DecisionTreeClassifier(criterion = "gini", splitter = "random")
num_trees = 1000
  
# bagging classifier
model = BaggingClassifier(base_estimator = base_cls,
                          n_estimators = num_trees,
                          random_state = seed).fit(X,y)
  
results = model_selection.cross_val_score(model, X, y, cv = kfold)
print("accuracy :")
print(results.mean())

### Predicting the values for the testing dataset and storing them into results.xlsx

In [ ]:
df_test = df_test.rename(columns={'iframe': 'Iframe'})
X_test_data = df_test.iloc[:, 1:] 

In [ ]:
y_pred = model.predict(X_test_data)

In [ ]:
final_df = pd.DataFrame(y_pred, index = None)

In [ ]:
final_df.to_excel("results.xlsx", "df1")